In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import shapefile
import pyproj


#! /usr/bin/env python3

# data libraries
import sys
import os
import datetime as dt  # Python standard library datetime  module
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
from netCDF4 import Dataset,num2date  # http://code.google.com/p/netcdf4-python/
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
from matplotlib.path import Path

# plotting libraries
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.style as style
from matplotlib import cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point 
import cartopy.mpl.ticker as cticker
import glob
import pickle

In [2]:
def getPolyCoords(row, geom, coord_type):
    """
    Returns the coordinates ('x|y') of edges/vertices of a Polygon/others
    Args:
    - row: the row object from a geodataframe; i.e.   df.loc[1]
    - geom: the name of "geometry" column, usually "geometry"
    - coord_type: 'x', or 'y'
    ...
    Valid geometrie types 'Polygon', 'Point', 'LineString', 'MultiLineString', 'MultiPolygon'
    """

    # Parse the geometries and grab the coordinate
    geometry = row[geom]
    #print(geometry.type)

    if geometry.type=='Polygon':
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            # Interior is more complex: xxx.interiors[0].coords.xy[0]
            return list( geometry.exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( geometry.exterior.coords.xy[1] )

    if geometry.type in ['Point', 'LineString']:
        if coord_type == 'x':
            return list( geometry.xy[0] )
        elif coord_type == 'y':
            return list( geometry.xy[1] )

    if geometry.type=='MultiLineString':
        all_xy = []
        # updated code, using .geoms
        for ea in geometry.geoms:
            if coord_type == 'x':
                all_xy.append(list( ea.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.xy[1] ))
        return all_xy

    if geometry.type=='MultiPolygon':
        all_xy = []
        for ea in geometry.geoms:
            if coord_type == 'x':
                all_xy.append(list( ea.exterior.coords.xy[0] ))
            elif coord_type == 'y':
                all_xy.append(list( ea.exterior.coords.xy[1] ))
        return all_xy

    else:
        # Finally, return empty list for unknown geometries
        return []

### Code to plot the Shape File Boundary

In [3]:
shelf = gpd.read_file('/raid01/mafields/tas/MODELS_filtered/ssp585/shape_files/Moscow University.shp') 
shelf.crs = "EPSG:3031"
shelf = shelf.to_crs(epsg=4326)

shelf_lon = getPolyCoords(shelf.loc[0], 'geometry', 'x')
shelf_lat = getPolyCoords(shelf.loc[0], 'geometry', 'y')
shelf_list = np.vstack([shelf_lat[0],shelf_lon[0]]).T
shelf_path = Path(shelf_list)

#plt.plot(shelf_lat,shelf_lon)

In [63]:
def cropping_function(path,path1):
# Lists of data pulled from paths to create new csv of cropped data
    fpath = list() # The path to location in my raid directory
    model_names = list() # The model name pulled from the file name of the data 
    modeltemps = list() # The temperature of the shelf for each year from model
    latitude = list()
    longitude = list()
    year = list()
    month = list()
    gridsize = list()
    temp_crop = list()
    newlat = list()
    newlon = list()
    spatial_avg = list()
    
    shelf = gpd.read_file(f'{path1}') 
    shelf.crs = "EPSG:3031"
    shelf = shelf.to_crs(epsg=4326)

    shelf_lon = getPolyCoords(shelf.loc[0], 'geometry', 'x')
    
    shelf_lon = np.array(shelf_lon)
    #shelf_lon = shelf_lon - 360*((shelf_lon)-np.abs(shelf_lon))/(2*np.abs(shelf_lon)) #This line should only be used for Ross ICe Shelf
    
    shelf_lat = getPolyCoords(shelf.loc[0], 'geometry', 'y')
    shelf_list = np.vstack([shelf_lat,shelf_lon]).T
    #shelf_list = np.vstack([shelf_lat[0],shelf_lon[0]]).T
    shelf_path = Path(shelf_list)
    
    for file in glob.iglob(f'{path}/*.nc'): #iterating through files in path specified
        
        #Reading in Datafiles
        fpath.append(file)
        fname = file
        datain = Dataset(fname, 'r')
        
        ##Splitting The Path to label the models
        head_tail = os.path.split(file) #makes the path a variable
        mystring = str(head_tail[1]) #turns the variable into a string
        x = mystring.split(".", 5)#parsing the model name by periods
        model_names.append(x[1]) # the second section of string is the model name
        print(model_names)
        #Creating Variables
        var = datain.variables['tas'] #2meter surface temperture
        var = var[:,:,:] - 273.15 #converting the temperature from Kelvin to Celsius
        modeltemps.append(var)
        grid = var[1].size
        gridsize.append(grid)
        lat = datain.variables['latitude'][:] #latitude
        latitude.append(lat)
        lon = datain.variables['longitude'][:] #longitude
        longitude.append(lon)
        timevar = datain.variables['time'] #in days since 1850
        dtime = num2date(timevar[:],timevar.units) # The next two lines make the time variable readable
        ym = np.zeros((len(dtime),2), int)
        
        shelf_lon_model, shelf_lat_model = np.meshgrid(lon,lat)
        inds = shelf_path.contains_points(np.vstack([shelf_lat_model.ravel(), shelf_lon_model.ravel()]).T)
        
        i = 0
        
        newvar = list()
        var_spatial_avg = list()
        for i in range(len(var)):
            vartemp = var[i].reshape((1,len(inds)))
            newvar.append(vartemp[0][inds])
            var_avg = np.mean(vartemp[0][inds])
            var_spatial_avg.append(var_avg)
        
        temp_crop.append(newvar)
        spatial_avg.append(var_spatial_avg)
        
            
        lat_temp = shelf_lat_model.ravel()[inds]
        lon_temp = shelf_lon_model.ravel()[inds]
        newlat.append(lat_temp)
        newlon.append(lon_temp)
        

        ##Time Component
        for i in range(len(dtime)): #Creating an aray of years from the time variable
            ym[i,0] = dtime[i].year
            ym[i,1] = dtime[i].month
       
        year.append(ym[:,0])
        month.append(ym[:,1])
    
    df = pd.DataFrame(list(zip(fpath,model_names,gridsize,modeltemps,year,month,latitude,longitude,temp_crop,newlat,newlon,spatial_avg)),
                      columns = ['file path', 'Model','gridsize','temp','year','month','original latitude','original longitude', 'cropped temp','cropped lat',
                                 'cropped lon','temp spatial average'])
    
    #df.to_csv(f'{path}/cropped/monthly.models.cropped.abbot.csv') 
    
    return df

In [5]:
rootdir = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/amery'
shapefile = '/raid01/mafields/tas/MODELS_filtered/ssp585/shape_files/Amery.shp'

test = cropping_function(rootdir,shapefile)

print(test)

Empty DataFrame
Columns: [file path, Model, gridsize, temp, year, month, original latitude, original longitude, cropped temp, cropped lat, cropped lon, temp spatial average]
Index: []


In [6]:
def reanalysis_cropping_function(path,path1):
# Lists of data pulled from paths to create new csv of cropped data
    fpath = list() # The path to location in my raid directory
    model_names = list() # The model name pulled from the file name of the data 
    modeltemps = list() # The temperature of the shelf for each year from model
    latitude = list()
    longitude = list()
    year = list()
    month = list()
    gridsize = list()
    temp_crop = list()
    newlat = list()
    newlon = list()
    spatial_avg = list()
    
    shelf = gpd.read_file(f'{path1}') 
    shelf.crs = "EPSG:3031"
    shelf = shelf.to_crs(epsg=4326)

    shelf_lon = getPolyCoords(shelf.loc[0], 'geometry', 'x')
    
    shelf_lon = np.array(shelf_lon)
    shelf_lon = shelf_lon - (360*((shelf_lon)-np.abs(shelf_lon))/(2*np.abs(shelf_lon))) #This line should only be used for Ross ICe Shelf
    
    shelf_lat = getPolyCoords(shelf.loc[0], 'geometry', 'y')
    shelf_list = np.vstack([shelf_lat,shelf_lon]).T
    #shelf_list = np.vstack([shelf_lat[0],shelf_lon[0]]).T
    shelf_path = Path(shelf_list)

    fname = path
    fpath.append(fname)
    datain = Dataset(fname, 'r')

    ##Splitting The Path to label the models
    head_tail = os.path.split(fname) #makes the path a variable
    mystring = str(head_tail[1]) #turns the variable into a string
    x = mystring.split(".", 5)#parsing the model name by periods
    model_names.append(x[1]) # the second section of string is the model name
    print(model_names)
    #Creating Variables
    var = datain.variables['t2m'] #2meter surface temperture
    var = var[:,:,:] - 273.15 #converting the temperature from Kelvin to Celsius
    modeltemps.append(var)
    grid = var[1].size
    gridsize.append(grid)
    lat = datain.variables['latitude'][:] #latitude
    latitude.append(lat)
    lon = datain.variables['longitude'][:] #longitude
    longitude.append(lon)
    timevar = datain.variables['time'] #in days since 1850
    dtime = num2date(timevar[:],timevar.units) # The next two lines make the time variable readable
    ym = np.zeros((len(dtime),2), int)

    shelf_lon_model, shelf_lat_model = np.meshgrid(lon,lat)
    inds = shelf_path.contains_points(np.vstack([shelf_lat_model.ravel(), shelf_lon_model.ravel()]).T)

    i = 0

    newvar = list()
    var_spatial_avg = list()
    for i in range(len(var)):
        vartemp = var[i].reshape((1,len(inds)))
        newvar.append(vartemp[0][inds])
        var_avg = np.mean(vartemp[0][inds])
        var_spatial_avg.append(var_avg)

    temp_crop.append(newvar)
    spatial_avg.append(var_spatial_avg)


    lat_temp = shelf_lat_model.ravel()[inds]
    lon_temp = shelf_lon_model.ravel()[inds]
    newlat.append(lat_temp)
    newlon.append(lon_temp)


    ##Time Component
    for i in range(len(dtime)): #Creating an aray of years from the time variable
        ym[i,0] = dtime[i].year
        ym[i,1] = dtime[i].month

    year.append(ym[:,0])
    month.append(ym[:,1])

    
    df = pd.DataFrame(list(zip(fpath,model_names,gridsize,modeltemps,year,month,latitude,longitude,temp_crop,newlat,newlon,spatial_avg)),
                      columns = ['file path', 'Model','gridsize','temp','year','month','original latitude','original longitude', 'cropped temp','cropped lat',
                                 'cropped lon','temp spatial average'])
 
    
    return df

In [7]:
monthly_shelves = {}
seasonal_shelves = {}
yearly_shelves = {}

In [12]:
#with open('saved_dictionary_monthly.pkl', 'rb') as f:
 #   monthly_shelves = pickle.load(f)

In [13]:
#with open('saved_dictionary_seasonal.pkl', 'rb') as f:
 #   seasonal_shelves = pickle.load(f)

In [14]:
#with open('saved_dictionary_yearly.pkl', 'rb') as f:
 #   yearly_shelves = pickle.load(f)

In [8]:
def iceshelf_dictionary(rootdir,shapefile,shelf):
    
    for subdir, dirs, files in os.walk(rootdir):
        path_list = list()
        for d in dirs:
            print(d)
            path = os.path.join(subdir, d)
            head_tail = os.path.split(path)
            df = cropping_function(path,shapefile)
            path_list.append(path)
            
            if head_tail[1] == 'seasonal' :
                seasonal_shelves[shelf] = df
                print('seasonal frame added')
            elif head_tail[1] == 'yearly': 
                yearly_shelves[shelf] = df
                print('yearly frame added')
            elif head_tail[1] == 'monthly': 
                monthly_shelves[shelf] = df
                print('monthly frame added')
                
    return path_list

In [182]:
seasonal_shelves.keys()

dict_keys(['Abbot', 'Amery', 'Atka', 'Baudouin', 'Borchgrevink', 'Brunt Stancomb', 'Conger Glenzer', 'Cook', 'Cosgrove', 'Crosson', 'Dotson', 'Drygalski', 'Ekstrom', 'Filchner', 'Fimbul', 'George VI', 'Getz', 'Holmes', 'Jelbart', 'Land', 'LarsenB', 'LarsenC', 'LarsenD', 'Lazarev', 'Mariner', 'Mertz', 'Moscow University', 'Nansen', 'Nickerson', 'Nivl', 'pine island', 'Prince Harald', 'Quar', 'Rennick', 'Riiser-Larsen', 'Ronne', 'Shackleton', 'Stange', 'Sulzberger', 'Thwaites', 'Totten', 'Venable', 'Vigrid', 'West', 'Withrow', 'Ross East', 'Ross West'])

In [74]:
yearly_shelves.keys()

dict_keys(['Abbot', 'Amery', 'Atka', 'Baudouin', 'Borchgrevink', 'Conger Glenzer', 'Cook', 'Cosgrove', 'Crosson', 'Dotson', 'Drygalski', 'Ekstrom', 'Filchner', 'George VI', 'Getz', 'Holmes', 'Jelbart', 'Land', 'LarsenB', 'LarsenC', 'LarsenD', 'Lazarev', 'Mariner', 'Mertz', 'Moscow University', 'Nansen', 'Nickerson', 'Nivl', 'pine island', 'Prince Harald', 'Quar', 'Rennick', 'Riiser-Larsen', 'Ronne', 'Ross East', 'Ross West', 'Shackleton', 'Stange', 'Sulzberger', 'Thwaites', 'Totten', 'Venable', 'Vigrid', 'West', 'Withrow'])

In [72]:
monthly_shelves.keys()

dict_keys(['Abbot', 'Amery', 'Atka', 'Baudouin', 'Borchgrevink', 'Brunt Stancomb', 'Conger Glenzer', 'Cook', 'Cosgrove', 'Crosson', 'Dotson', 'Drygalski', 'Ekstrom', 'Filchner', 'George VI', 'Getz', 'Holmes', 'Jelbart', 'Land', 'LarsenB', 'LarsenC', 'LarsenD', 'Lazarev', 'Mariner', 'Mertz', 'Moscow University', 'Nansen', 'Nickerson', 'Nivl', 'pine island', 'Prince Harald', 'Quar', 'Rennick', 'Riiser-Larsen', 'Ronne', 'Ross East', 'Ross West', 'Shackleton', 'Stange', 'Sulzberger', 'Thwaites', 'Totten', 'Venable', 'Vigrid', 'West', 'Withrow'])

In [73]:
rootdir = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves_attempt2/withrow'
shapefile = '/raid01/mafields/tas/MODELS_filtered/ssp585/shape_files/Withrow.shp'
shelf = 'Withrow'


pathlist  = iceshelf_dictionary(rootdir,shapefile,shelf)

monthly
['E3SM-1-1']
['E3SM-1-1', 'CAS-ESM2-0']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Proj

['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR', 'EC-Earth3']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR', 'EC-Earth3', 'IITM-ESM']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1', 'CMCC-CM2-SR5

['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR', 'EC-Earth3', 'IITM-ESM', 'NorESM2-MM', 'FGOALS-g3', 'ACCESS-ESM1-5', 'NorESM2-LM']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR', 'EC-Earth3', 'IITM-ESM', 'NorESM2-MM', 'FGOALS-g3', 'ACCESS-ESM1-5', 'NorESM2-LM', 'ACCESS-CM2']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'INM-CM5-0', 'FGOALS-f3-L', 'AWI-CM-1-1-MR', 'TaiESM1', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'IPSL-CM6A-LR', 'EC-Earth3', 'IITM-ESM', 'NorESM2-MM', 'FGOALS-g3', 'ACCESS-ESM1-5', 'NorESM2-LM', 'ACCESS-CM2', 'GFDL-ESM4']
['E3SM-1-1', 'CAS-ESM2-0', 'KIOST-ESM', 'E3SM-Project', 'CAMS-CSM1-0', 'MPI-ESM1-2-HR', 'INM-CM4-8', 'IN

In [98]:
reanalysis_monthly = {}
reanalysis_seasonal = {}
reanalysis_yearly = {}

In [94]:
import pickle

with open('reanalysis_monthly.pkl', 'rb') as f:
    monthly_reanalysis = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'reanalysis_monthly.pkl'

In [ ]:
with open('reanalysis_seasonal.pkl', 'rb') as f:
    seasonal_reanalysis = pickle.load(f)

In [ ]:
with open('reanalysis_yearly.pkl', 'rb') as f:
    yearly_reanalysis = pickle.load(f)

In [96]:
def reanalysis_dictionary(path,path2,path3,shapefile,shelf):
    
        path_list = list()
        filler = path
        df_mon = reanalysis_cropping_function(path,shapefile)   
        reanalysis_monthly[shelf] = df_mon  
        
        df_seas = reanalysis_cropping_function(path2,shapefile)   
        reanalysis_seasonal[shelf] = df_seas
        
        df_year = reanalysis_cropping_function(path3,shapefile)   
        reanalysis_yearly[shelf] = df_year
        
        path_list.append(filler)
        
        return path_list

In [181]:
reanalysis_yearly.keys()

dict_keys(['Abbot', 'Amery', 'Atka', 'Baudouin', 'Borchgrevink', 'Brunt Stancomb', 'Conger Glenzer', 'Cook', 'Cosgrove', 'Crosson', 'Dotson', 'Drygalski', 'Ekstrom', 'Filchner', 'Fimbul', 'George VI', 'Getz', 'Holmes', 'Jelbart', 'Land', 'LarsenB', 'LarsenC', 'LarsenD', 'Lazarev', 'Mariner', 'Mertz', 'Moscow University', 'Nansen', 'Nickerson', 'Nivl', 'pine island', 'Prince Harald', 'Quar', 'Rennick', 'Riiser-Larsen', 'Ronne', 'Ross East', 'Ross West', 'Shackleton', 'Stange', 'Sulzberger', 'Thwaites', 'Totten', 'Venable', 'Vigrid', 'West', 'Withrow'])

,file path,Model,gridsize,temp,year,month,original latitude,original longitude,cropped temp,cropped lat,cropped lon,temp spatial average
0,/raid01/mafields/tas/MODELS_filtered/ssp585/ic...,seasmean,285,"[[[-2.472286931143344, -2.6843103070465872, -2...","[1940, 1941, 1942, 1943, 1944, 1945, 1946, 194...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-72.25, -72.5, -72.75, -73.0, -73.25]","[-103.25, -103.0, -102.75, -102.5, -102.25, -1...","[[-2.472286931143344, -2.6843103070465872, -3....","[-72.25, -72.25, -72.25, -72.25, -72.25, -72.2...","[-103.25, -103.0, -102.5, -102.25, -102.0, -10...","[-4.594308031595278, -4.432360681488522, -5.82..."


In [180]:
path = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/monthly/ERA5_reanalysis_update.withrow.nc'
path2 = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/seasavg/ERA5_reanalysis_update.seasmean.withrow.nc'
path3 = '/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/reanalysis/yearavg/ERA5_reanalysis_update.yearmean.withrow.nc'
shapefile = '/raid01/mafields/tas/MODELS_filtered/ssp585/shape_files/Withrow.shp'
shelf = 'Withrow'

pathlist  = reanalysis_dictionary(path,path2,path3,shapefile,shelf)

['withrow']
['seasmean']
['yearmean']


In [24]:
ross_spatial_avg = np.zeros((len(seasonal_shelves['Ross East']['temp spatial average']),len(seasonal_shelves['Ross East']['temp spatial average'][0])))

print(len(seasonal_shelves['Ross East']['temp spatial average'][1]),len(seasonal_shelves['Ross West']['temp spatial average'][1]))

for i in range(len(seasonal_shelves['Ross East']['temp spatial average'])):
    ross_spatial_avg[i] = (np.array(seasonal_shelves['Ross East']['temp spatial average'][i]) + np.array(seasonal_shelves['Ross West']['temp spatial average'][i]))/2
#ross_temp_avg = (seasonal_shelves['Ross West']['temp spatial average']+seasonal_shelves['Ross West']['temp spatial average'])/2
#Ross_gridsize = seasonal_shelves['Ross East']['temp spatial average'] + seasonal_shelves['Ross West']['temp spatial average']

252 252


ValueError: could not broadcast input array from shape (251,) into shape (252,)

In [17]:
for subdir, dirs, files in os.walk(rootdir):
    for d in dirs:
        path = os.path.join(subdir, d)
        head_tail = os.path.split(path)
        df = cropping_function(path,shapefile)
        if head_tail[1] == 'seasonal' :
            print(path)
        elif head_tail[1] == 'yearly': 
            print(path)
        elif head_tail[1] == 'monthly': 
            print(path)

/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/abbot/seasonal
/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/abbot/yearly
/raid01/mafields/tas/MODELS_filtered/ssp585/iceshelves/abbot/monthly


In [75]:
import pickle 

with open('monthly_shelves.pkl', 'wb') as f:
    pickle.dump(monthly_shelves, f)

In [76]:
with open('yearly_shelves.pkl', 'wb') as f:
    pickle.dump(yearly_shelves, f)

In [77]:
with open('seasonal_shelves.pkl', 'wb') as f:
    pickle.dump(seasonal_shelves, f)

In [185]:
with open('reanalysis_monthly.pkl', 'wb') as f:
    pickle.dump(reanalysis_monthly, f)

In [186]:
with open('reanalysis_yearly.pkl', 'wb') as f:
    pickle.dump(reanalysis_yearly, f)

In [187]:
with open('reanalysis_seasonal.pkl', 'wb') as f:
    pickle.dump(reanalysis_seasonal, f)